# Lectura y balanceo de datos

## Lectura

In [19]:
import csv
import numpy as np

#Nombre del fichero que quiero leer
datos = "creditcard.csv"

#Inicialización de los arrays
caracteristicas = []
clase = []

#Lectura de datos
with open(datos) as f:
    for i, linea in enumerate(f):
        #Saltamos la cabecera del csv
        if i == 0:
            print("Cabecera:", linea.strip())
            continue  
        #Introducimos los datos en los arrays de características
        campos = linea.strip().split(",")
        caracteristicas.append([float(v.replace('"', "")) for v in campos[:-1]])
        clase.append([int(campos[-1].replace('"', ""))])
        if i == 1:
            print("Ejemplo de caracteristicas:", caracteristicas[-1])

#Conversión de datos
data = np.array(caracteristicas, dtype="float32")
target = np.array(clase, dtype="uint8")

#Pintamos la forma de los datos
print("Forma de los datos de entrada al modelo:", data.shape)
print("Forma de las clases de salida:", target.shape)

Cabecera: "Time","V1","V2","V3","V4","V5","V6","V7","V8","V9","V10","V11","V12","V13","V14","V15","V16","V17","V18","V19","V20","V21","V22","V23","V24","V25","V26","V27","V28","Amount","Class"
Ejemplo de caracteristicas: [0.0, -1.3598071336738, -0.0727811733098497, 2.53634673796914, 1.37815522427443, -0.338320769942518, 0.462387777762292, 0.239598554061257, 0.0986979012610507, 0.363786969611213, 0.0907941719789316, -0.551599533260813, -0.617800855762348, -0.991389847235408, -0.311169353699879, 1.46817697209427, -0.470400525259478, 0.207971241929242, 0.0257905801985591, 0.403992960255733, 0.251412098239705, -0.018306777944153, 0.277837575558899, -0.110473910188767, 0.0669280749146731, 0.128539358273528, -0.189114843888824, 0.133558376740387, -0.0210530534538215, 149.62]
Forma de los datos de entrada al modelo: (284807, 30)
Forma de las clases de salida: (284807, 1)


## Balanceo 

In [20]:
#Primero analizamos los datos

#Inicializacion de la cuenta
legal = 0
fraude = 0

#Cuenta de datos
for x in range(target.shape[0]):
    if target[x] == 0:
        legal = legal + 1
    else:
        fraude = fraude + 1

#Representación
print("Tarjetas legales: " + str(legal))
print("Tarjetas fraudulentas: " + str(fraude))

Tarjetas legales: 284315
Tarjetas fraudulentas: 492


In [21]:
#Sobremuestreamos por la gran diferencia

#Importamos los paquetes de sobremuestreo
from imblearn.over_sampling import SMOTE

#SMOTE
smote = SMOTE()

#Generación de nuevas muestras sintéticas
dataSmote, targetSmote = smote.fit_resample(data,target)

#Volvemos a contar
legal = 0
fraude = 0

#Cuenta de datos
for x in range(targetSmote.shape[0]):
    if targetSmote[x] == 0:
        legal = legal + 1
    else:
        fraude = fraude + 1

#Representación
print("Tarjetas legales balanceadas: " + str(legal))
print("Tarjetas fraudulentas balanceadas: " + str(fraude))

Tarjetas legales balanceadas: 284315
Tarjetas fraudulentas balanceadas: 284315


In [22]:
#División de datos en conjunto de evaluación y conjunto de entrenamiento
from sklearn.model_selection import train_test_split
dataTrain, dataTest, targetTrain, targetTest = train_test_split(dataSmote,targetSmote, random_state = 0)

In [23]:
#Normalización de los datos 

#Cálculo de la media
mean = np.mean(dataTrain, axis=0)

#Restamos a las características la media
dataTrain -= mean
dataTest -= mean

#Cálculo de la desviación estándar
std = np.std(dataTrain, axis=0)

#Dividimos entre la desviación estándar
dataTrain /= std
dataTest /= std

# Entrenamiento y evaluación de modelos

## Construcción del modelo con Keras

In [24]:
from tensorflow import keras

#Añadimos las capas de nuestra red neuronal (3 densas y dos de dropout)
#Otra opcion seria añadirlas con add (próximo video)
model = keras.Sequential(
    [
        #Capa densa, la primera capa siempre tiene que especificar la forma de entrada
        keras.layers.Dense(
            256, activation="relu", input_shape=(dataTrain.shape[-1],) #Nodos de la capa densa, y función de activación
        ),
        keras.layers.Dense(256, activation="relu"),
        #Capa de Dropout. Inactiva algunos de los nodos de la red para evitar el sobreentrenamiento
        keras.layers.Dropout(0.3), #El atributo que se pone es el ratio de inactivación
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(1, activation="sigmoid"),
    ]
)

#Vemos la forma de nuestro modelo
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 256)               7936      
_________________________________________________________________
dense_9 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 256)               65792     
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 257       
Total params: 139,777
Trainable params: 139,777
Non-trainable params: 0
________________________________________________

## Entrenamiento y evaluación del modelo

In [25]:
#Funciones de las métricas (sacadas de internet: https://datascience.stackexchange.com/questions/45165/how-to-get-accuracy-f1-precision-and-recall-for-a-keras-model)
from keras import backend as K

#Funciones de las métricas
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [26]:
#Compilamos el modelo
model.compile(
    optimizer=keras.optimizers.Adam(1e-2), loss="binary_crossentropy", metrics=['accuracy',f1_m,precision_m, recall_m]
) #Compilamos el modelo con su optimizador, la forma en la que actualizamos los pesos (minimización) y en base a qué métricas

#Lo entrenamos con los datos de entrenamiento
model.fit(
    dataTrain,
    targetTrain,
    batch_size=2048,
    epochs=30,
    verbose=1,
    validation_data=(dataTest, targetTest),
)#Conjuntos de entrenamientos y evaluación, numero de muestras en la propagación hacia atrás, 
#numero de iteraciones para mejorar el modelo, la verbosidad y los conjuntos de validación

Train on 426472 samples, validate on 142158 samples
Epoch 1/30
426472/426472 [==============================] - 31s 73us/sample - loss: 0.0448 - acc: 0.9839 - f1_m: 0.9844 - precision_m: 0.9878 - recall_m: 0.9819 - val_loss: 0.0089 - val_acc: 0.9974 - val_f1_m: 0.9974 - val_precision_m: 0.9976 - val_recall_m: 0.9972
Epoch 2/30
426472/426472 [==============================] - 29s 67us/sample - loss: 0.0084 - acc: 0.9977 - f1_m: 0.9977 - precision_m: 0.9970 - recall_m: 0.9984 - val_loss: 0.0040 - val_acc: 0.9990 - val_f1_m: 0.9990 - val_precision_m: 0.9980 - val_recall_m: 1.0000
Epoch 3/30
426472/426472 [==============================] - 31s 73us/sample - loss: 0.0066 - acc: 0.9984 - f1_m: 0.9984 - precision_m: 0.9978 - recall_m: 0.9990 - val_loss: 0.0037 - val_acc: 0.9992 - val_f1_m: 0.9992 - val_precision_m: 0.9984 - val_recall_m: 0.9999
Epoch 4/30
426472/426472 [==============================] - 32s 74us/sample - loss: 0.0052 - acc: 0.9987 - f1_m: 0.9987 - precision_m: 0.9981 - recall

In [28]:
#Métricas
#Conjunto de entrenamiento
print("Datos sobre el entrenamiento")
loss, accuracy,f1_score, precision, recall = model.evaluate(dataTrain, targetTrain, verbose=False)
print("Exactitud de entrenamiento: {:.4f}".format(accuracy))
print("F1 de entrenamiento: {:.4f}".format(f1_score))
print("Precisión de entrenamiento: {:.4f}".format(precision))
print("Memoria de entrenamiento: {:.4f}".format(recall))

#Conjunto de evaluación
print()
print("Datos sobre la evaluación")
loss, accuracy,f1_score, precision, recall = model.evaluate(dataTest, targetTest, verbose=False)
print("Exactitud de evaluación: {:.4f}".format(accuracy))
print("F1 de evaluación: {:.4f}".format(f1_score))
print("Precisión de evaluación: {:.4f}".format(precision))
print("Memoria de evaluación: {:.4f}".format(recall))